In [1]:
from functions import sample_hyperparameters, get_features, discretize_trajectories, get_feature_traj_paths, bootstrap_vamp
import pickle
import numpy as np
import pyemma as pm
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from pathlib import Path
from rich.progress import Progress

In [2]:
search_space = pickle.load(open('common/hp_search_space.p', 'rb'))
features = ["contact_filtered", 'alpha', 'phi_psi']
sim_names = ['transition_state', 'reactant']
lag = 400
score_k = 2+1

# Introduction

Having established a lag time of 40ns and 3 dominant (2 non-trivial) relaxation processes to score in the previous notebooks. This notebook will score randomly sampled hyperparameters, this will be the same as 5a, but with more hps per feature. 

In [ ]:
np.random.seed(93475)

hps_per_feature = 30
n_bootstraps = 100
failed_hps = []

for sim in sim_names:
    for feature in features:
        
        traj_paths = get_feature_traj_paths(sim, feature)
        with Progress() as progress:
            
            task = progress.add_task('[green] Sampling {2} HPs for {0} with {1} feature'.format(sim, feature, hps_per_feature), total=hps_per_feature)

            for i in range(hps_per_feature):

                hps = sample_hyperparameters(search_space)
                fname = f"{sim}_{feature}_l{hps['vamp']['lag']}_d{hps['vamp']['dim']}_n{hps['cluster_kmeans']['k']}.p"
                fpath = join('hp_sampling', fname)
                if not Path(fpath).exists():
                    try:
                        vamps = bootstrap_vamp(n=n_bootstraps, hps=hps, traj_paths=traj_paths, lag=lag, vamp_k=score_k)
                        pickle.dump(file=open(fpath, 'wb'), obj=[hps, vamps])
                        progress.update(task, advance=1)
                    except: 
                        print('ERROR with ', fname)
                        failed_hps.append(hps)

In [16]:
# hps_per_feature = 5
n_bootstraps = 100
sim = 'reactant'
feature = 'phi_psi'
hps = sample_hyperparameters(search_space)
print(hps)
fname = f"{sim}_{feature}_l{hps['vamp']['lag']}_d{hps['vamp']['dim']}_n{hps['cluster_kmeans']['k']}.p"
fpath = join('lag_determination', fname)
traj_paths = get_feature_traj_paths(sim, feature)

bs_its = bootstrap_ts2(n=n_bootstraps, hps=hps, traj_paths=traj_paths, lags=lags)


pickle.dump(file=open(fpath, 'wb'), obj=[hps, bs_its])


{'vamp': {'dim': 5, 'lag': 144}, 'cluster_kmeans': {'k': 103, 'max_iter': 1000}}
8
